In [ ]:
import json
import os
import pandas
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [ ]:
metrc_download_summary_companies_query = create_queries.create_metrc_download_summary_companies_query()
metrc_download_summary_companies_dataframe = pandas.read_sql_query(metrc_download_summary_companies_query, engine)

In [ ]:
company_records = metrc_download_summary_companies_dataframe.to_dict('records')

In [ ]:
company_to_retailer_licenses = {}

for company_record in company_records:
    company_identifier = company_record['identifier']
    company_name = company_record['name']
    print(f'Checking if company {company_identifier} holds any Retailer licenses...')
    company_licenses_query = create_queries.create_company_licenses_query(company_identifier)
    company_licenses_dataframe = pandas.read_sql_query(company_licenses_query, engine)
    company_license_records = company_licenses_dataframe.to_dict('records')
    for company_license_record in company_license_records:
        license_number = company_license_record['license_number']
        license_category = company_license_record['license_category']
        key = (company_identifier, company_name)
        if license_category == 'Retailer':
            if key not in company_to_retailer_licenses:
                company_to_retailer_licenses[key] = []
            company_to_retailer_licenses[key].append(license_number)

In [ ]:
for company_tuple, retailer_licenses in company_to_retailer_licenses.items():
    company_identifier, company_name = company_tuple
    print(f'Company {company_name} ({company_identifier}) holds Retailer licenses: {retailer_licenses}')

In [ ]:
len(company_to_retailer_licenses.keys())

In [ ]:
# Output to copy to XLSX
retailer_company_tuples = []

for company_tuple, retailer_licenses in company_to_retailer_licenses.items():
    company_identifier, company_name = company_tuple
    company_licenses = ";".join(retailer_licenses)
    retailer_company_tuples.append((company_name, company_identifier, company_licenses))

In [ ]:
retailer_companies_dataframe = pandas.DataFrame(retailer_company_tuples)
retailer_companies_dataframe

In [ ]:
import time
from datetime import date

TODAY_DATE = date.today().strftime('%m-%d-%Y')
NOW = int(time.time())

retailer_companies_file_name = f'~/Downloads/retailer_companies_file_name_{TODAY_DATE}_{NOW}'

num_retailer_companies = len(retailer_companies_dataframe.index)
print(f'Exporting {num_retailer_companies} retailer companies to files...')
retailer_companies_dataframe.reset_index().to_excel(f'{retailer_companies_file_name}.xlsx')
# export_incoming_transfer_packages_dataframe.reset_index().to_csv(f'{incoming_transfer_packages_file_name}.csv')
print(f'Exported {num_retailer_companies} retailer companies to files')